In [1]:
import numpy as np 
import pandas as pd 
import os 
import plotly.graph_objects as go


In [2]:
Shayya_bk_rnaseq = './files/Shayya/bk_rnaseq/'
DE_list = [i for i in os.listdir(Shayya_bk_rnaseq) if 'DE' in i]

In [128]:
DE_df = {}
for DE in DE_list: 
    DE_col = DE.split('.')[0]
    temp = pd.read_table(Shayya_bk_rnaseq+DE, index_col=0)[['log2FoldChange', 'padj']].dropna()
#     temp.columns = [DE_col+' log2FoldChange', DE_col+' padj']
    DE_df[DE] = temp
#     DE_df = pd.concat([DE_df, temp], axis=1)

In [ ]:
# Grabs index(gene name) from hOE dataframe and create separate dataframe with full gene name
import mygene
mg = mygene.MyGeneInfo()

gene_list = []
for DE in DE_df: 
    gene_list += list(DE_df[DE][DE_df[DE]['padj'] < 0.05].index)
#     line below removes duplicate genes in the list 
    gene_list = [*set(gene_list)]
        
        
gene_name_list = pd.DataFrame(index = gene_list, columns=['gene_name'])
# gene_name_list = pd.read_csv('./output/Shayya/gene_name_list.csv', index_col= 0)
for gene in gene_name_list.index:
    gene_name = mg.query(gene)['hits']
    if len(gene_name) != 0: 
        try:
            gene_name_list = gene_name_list.append(pd.DataFrame([[gene_name[0]['name']]],
                                               columns = ['gene_name'], 
                                               index=[gene]))
            print(gene, end = ' ')
        except:
            print(gene + "NOT FOUND")
            gene_name_list = gene_name_list.append(pd.DataFrame([['']],
                                               columns = ['gene_name'], 
                                               index=[gene]))

gene_name_list.to_csv('./output/Shayya/gene_name_list.csv')

In [312]:
# Updates the dataframe with additional gene_name
for DE in DE_df: 
    temp = pd.concat([DE_df[DE], gene_name_list], join='outer', axis=1)
    DE_df[DE] = temp.dropna(subset=['log2FoldChange','padj'])

In [2]:
# Save labeled DE_df dictionary for future 
import pickle 
# Save dicitonary 
# with open("./output/Shayya/DE_df.csv", "wb") as f:
#     pickle.dump(DE_df, f)
# Read dictionary
with open("./output/Shayya/DE_df.csv", "rb") as f:
    DE_df = pickle.load(f)

In [15]:
mOSN_markers = ['Adcy3','Ano2', 'Cnga2', 'Cnga4', 'Cngb1', 'Gnal', 'Gng13', 
                  'Omp','Ric8b','Stom', 'Stoml3']
UPR_markers = ['Atf5', 'Nrp1', 'Sema3a']
cloned_genes = ['Rtp1', 'Rtbdn','Ebf4','Lingo2','Flrt1','Gpx6','Jph4','Efna3','Hspa12a','Sult1d1','Acsl6','Cfap206','Pde6d','Slc24a2','Ttc21a','Fam217a','Herpud1','Tmbim6','Manea','Clstn2','Ccdc184','Ebf2']
neg_21minus = ['Rtp1', 'Gfy', 'Clgn', 'Hspa12a', 'Ccdc184', 'Efna3', 'Clstn2', 'Jph4','Rtbdn']
pos_21minus = ['Cfap206','Lingo2','Acsl6','Sult1d1','Fam217a','Pde6d','Ebf2','Ebf4','Flrt1','Manea','Ttc21a']

In [ ]:

for DE in DE_df:
    data = DE_df[DE]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data[data['padj'] > 0.05]['log2FoldChange'], 
                            y=-np.log10(data[data['padj'] > 0.05]['padj']),
                            text=data[data['padj'] > 0.05].index,
                            mode='markers', 
                            name='padj > 0.05',
                            marker=dict(color = 'grey', opacity=0.1)))
    # plot padj < 0.1
    fig.add_trace(go.Scatter(x=data[data['padj'] < 0.05]['log2FoldChange'], 
                            y=-np.log10(data[data['padj'] < 0.05]['padj']),
                            text=data[data['padj'] < 0.05].index + "\n" +
                                  data[data['padj'] < 0.05].gene_name,
                            mode='markers',
                            name='padj < 0.05',
                            marker=dict(color = 'red',
                                       size = 10, opacity = 0.3)))
    # # plot OR genes
    # fig.add_trace(go.Scatter(x=data[list(data.index.str.startswith('OR'))]['log2FoldChange'], 
    #                         y=-np.log10(data[list(data.index.str.startswith('OR'))]['padj']),
    #                         text=data[list(data.index.str.startswith('OR'))].index,
    #                         mode='markers', 
    #                         name='Olfactory Receptors',
    #                         marker=dict(color = 'purple')))
    # Add marker genes 
    fig.add_trace(go.Scatter(x=data[data.index.isin(pos_21minus)]['log2FoldChange'], 
                            y=-np.log10(data[data.index.isin(pos_21minus)]['padj']),
                            text=data[data.index.isin(pos_21minus)].index,
    #                         textposition='top center',
                            mode='markers', 
                            name='pos_21minus',
                            marker=dict(color = 'purple',
                                       size = 10)))
    fig.add_trace(go.Scatter(x=data[data.index.isin(neg_21minus)]['log2FoldChange'], 
                            y=-np.log10(data[data.index.isin(neg_21minus)]['padj']),
                            text=data[data.index.isin(neg_21minus)].index,
    #                         textposition='top center',
                            mode='markers', 
                            name='neg_21minus',
                            marker=dict(color = 'green',
                                       size = 10)))



    # Adding intercept lines 
    fig.add_vline(x=0, line_width = 1, line_dash='dash')
    fig.add_hline(y=-np.log10(0.05), line_width = 1, line_dash='dash')
    # Scaling x axis by log2fc max and min 
    fig.update_xaxes(range=[min(data['log2FoldChange'])*1.1,
                            max(data['log2FoldChange'])*1.1])
    # fig.update_yaxes(range=[min(-np.log10(data['padj']))*1.1,
    #                         max(-np.log10(data['padj']))*1.1])
    fig.update_traces( 
        textposition='top center',
        hovertemplate =
        '<b>%{text}</b>' + 
        '<br>LogFC: %{x}'+
        '<br>-log10(padj): %{y}<br>')
    fig.update_layout(
        title=DE.split('.')[0],
        autosize=False,
        width=800,
        height=800,
        template='simple_white'
    )
    fig.show()
    
#     break
#     fig.write_html('./output/Shayya/bkRNAseq_vol/'+DE.split('.')[0]+'.html')

In [106]:
data = DE_df['DE_iRFPBright_vs_Dim.tsv']

In [38]:
uOR = pd.read_csv('./files/Shayya/bk_rnaseq/uOR_Matsunami.csv')
oOR = pd.read_csv('./files/Shayya/bk_rnaseq/oOR_Matsunami.csv')
uOR['Rtp_dependence'] = 'uOR'
oOR['Rtp_dependence'] = 'oOR'

Plot DE bright vs Dim by coloring uOR and oOR. 
<br>
There are some ORs that are not classified as either uOR or oOR, hence they will not be labeled 

In [111]:
data = DE_df['DE_iRFPBright_vs_Dim.tsv']

# Subset data 
OR_data = data[data.index.isin(OR_dependence_list.id)]
OR_data.loc[OR_data.index.isin(uOR.id), ['Rtp_dependence']] = 'uOR'
OR_data.loc[OR_data.index.isin(oOR.id), ['Rtp_dependence']] = 'oOR'
data = data[data.index.isin(OR_dependence_list.id) == False]


fig = go.Figure()
fig.add_trace(go.Scatter(x=data['log2FoldChange'], 
                        y=-np.log10(data['padj']),
                        text=data.index,
                        mode='markers', 
                        marker=dict(color = 'grey', opacity=0.1)))

# Plot oOR and uOR
fig.add_trace(go.Scatter(x=OR_data[OR_data['Rtp_dependence']=='uOR']['log2FoldChange'], 
                        y=-np.log10(OR_data[OR_data['Rtp_dependence']=='uOR']['padj']),
                        text=OR_data[OR_data['Rtp_dependence']=='uOR'].index,
#                         textposition='top center',
                        mode='markers', 
                        name='uOR',
                        marker=dict(color = 'purple',
                                    opacity=0.5,
                                   size = 10)))
fig.add_trace(go.Scatter(x=OR_data[OR_data['Rtp_dependence']=='oOR']['log2FoldChange'], 
                        y=-np.log10(OR_data[OR_data['Rtp_dependence']=='oOR']['padj']),
                        text=OR_data[OR_data['Rtp_dependence']=='oOR'].index,
#                         textposition='top center',
                        mode='markers', 
                        name='oOR',
                        marker=dict(color = 'blue',
                                    opacity=0.5,
                                   size = 10)))



# Adding intercept lines 
fig.add_vline(x=0, line_width = 1, line_dash='dash')
fig.add_hline(y=-np.log10(0.05), line_width = 1, line_dash='dash')
# Scaling x axis by log2fc max and min 
fig.update_xaxes(range=[min(data['log2FoldChange'])*1.1,
                        max(data['log2FoldChange'])*1.1])
# fig.update_yaxes(range=[min(-np.log10(data['padj']))*1.1,
#                         max(-np.log10(data['padj']))*1.1])
fig.update_traces( 
    textposition='top center',
    hovertemplate =
    '<b>%{text}</b>' + 
    '<br>LogFC: %{x}'+
    '<br>-log10(padj): %{y}<br>')
fig.update_layout(
    title='DE_iRFPBright_vs_Dim uOR oOR distribution',
    autosize=False,
    width=800,
    height=800,
    template='simple_white'
)
fig.show()
    
#     break
# fig.write_html('./output/Shayya/bkRNAseq_vol/'+DE.split('.')[0]+'_uoOR.html')

Output all DE_df to csv 


In [5]:
# for DE in DE_df:
#     DE_df[DE].to_csv('./output/Shayya/'+DE.split('.')[0]+'.csv')

 Assign rank orders for the log2FoldChange of iRFP bright vs dim. Combine the ranking by gene index with Cell_gene_ranks. This will allow us to narrow down genes that are expressed in high/low stress that is specific to mOSN

In [18]:
# ge_table = pd.read_csv('./output/brann/ge_table_ALL_Olfr_normExp_byCluster.csv', index_col=[0])
# test = ge_table.loc[['iOSN']].transpose().sort_values('iOSN', ascending=False)
# test['rank'] = list(range(len(ge_table.columns),0,-1))

In [15]:
msvipter_df = pd.read_table('./files/Shayya/aracneviper/msviper_out.tsv')[['Regulon','NES_avg','fdr']]
DE_stress_df = pd.read_csv('./output/Shayya/bkRNAseq_csv/DE_iRFPBright_vs_Dim.csv', index_col=0)[['log2FoldChange', 'padj']]
cell_gene_rank = pd.read_csv('./output/brann/CELL_genes_rank_include_Rik_GMgenes.csv', index_col=0)
mse_rtp1_df = pd.read_csv('./expression_csv/mse_normalized_rtp1_GSE151346_MOE_ALL_OlfrSeperated_include_Rik_GMgenes.csv', index_col=0)
mse_calr_df = pd.read_csv('./expression_csv/mse_normalized_Calr_GSE151346_MOE_ALL_OlfrSeperated_include_Rik_GMgenes.csv', index_col=0)

In [38]:
# assign a new column for if the gene is expressed more in high or low stress cells 
DE_stress_df = DE_stress_df.assign(bkRNA_Stress = ['high' if log2FoldChange > 0 else 'low' for log2FoldChange in DE_stress_df['log2FoldChange']])
DE_stress_df.sort_values('log2FoldChange', ascending=False, inplace=True)
DE_stress_df = DE_stress_df.assign(bkRNA_Rank = list(range(1,len(DE_stress_df)+1)))
# assign a new column for if the tf is expressed more in high or low stress cells 
msvipter_df = msvipter_df.assign(msVIPER_Stress = ['high' if NES_avg > 0 else 'low' for NES_avg in msvipter_df['NES_avg']])
msvipter_df.sort_values('NES_avg', ascending=False, inplace=True)
msvipter_df = msvipter_df.assign(msVIPER_Rank = list(range(1,len(msvipter_df)+1)))
msvipter_df.set_index('Regulon', inplace=True)
# assign new columns for mse ranks 
# Combine mse_df for rtp1 and calr, also change mse value to ranks insteads 
mse_rtp1_df.index = ['Rtp1_mse']
mse_calr_df.index = ['Calr_mse']
mse_df = pd.concat([mse_rtp1_df.transpose(), mse_calr_df.transpose()], axis=1)
mse_df.sort_values('Rtp1_mse', ascending=True, inplace=True)
mse_df = mse_df.assign(Rtp1_mse_Rank = list(range(1,len(mse_df)+1)))
mse_df.sort_values('Calr_mse', ascending=True, inplace=True)
mse_df = mse_df.assign(Calr_mse_Rank = list(range(1,len(mse_df)+1)))
mse_df = mse_df[['Rtp1_mse_Rank','Calr_mse_Rank']]

In [42]:
Stress_Rank_df = pd.concat([cell_gene_rank,DE_stress_df], axis=1)
Stress_Rank_df = pd.concat([Stress_Rank_df,msvipter_df], axis=1)
Stress_Rank_df = pd.concat([Stress_Rank_df,mse_df], axis=1)
# Add Gene_name to gene index
gene_name_list = pd.read_csv('./output/Shayya/gene_name_list.csv', index_col=0)
Stress_Rank_df = pd.concat([Stress_Rank_df,gene_name_list], axis=1)
# Save file 
# Stress_Rank_df.to_csv('./output/Shayya/Stress_Rank_df_includeRikGMgenes.csv')
